In [ ]:
%run Utils/Functions/core

In [ ]:
import requests
import json
import concurrent.futures
import time
from uuid import uuid4
import sys
import os
import re
from time import sleep, mktime
import json
from datetime import datetime, timedelta, timezone

In [ ]:
## Cria as variáveis de LOG
format_log = '%Y-%m-%d %H:%M:%S'
dtInicio = datetime.today() - timedelta(hours=3)
dtInicio_format = dtInicio.strftime(format_log)
tipo_log = 'API'
camada = 'landing'
emissor = '<org>'
atividade = 'activity_desc'
origem = 'RestService'
destino = 'AzureBlobFS'
execUrl = ' '

try:
    infos = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson()) # captura as informações do job que executa o notebook
    orgId = infos['tags']['orgId']
    runId = infos['tags']['multitaskParentRunId']
    jobId = infos['tags']['jobId']
    if orgId == '2960871991268730': # Monta a URL caso seja o ID do ambiente de DEV
        execUrl = f'https://adb-{orgId}.10.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
    else: # Monta a URL caso seja o ID do ambiente de PROD
        execUrl = f'https://adb-{orgId}.15.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
except:
    print('Campo de URL não pode ser identificado!')

In [ ]:
dbutils.widgets.text("dt_ingestao", "")

dt_ingestao = getArgument("dt_ingestao").upper().strip()

location_landing = spark.sql("show external locations").select("url").where("name = 'landing-area'").collect()[0][0]

In [ ]:
# Formata o dt_ingestao
format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
format_timestamp_api = '%Y-%m-%d'
dt_ingestao = datetime.now() if dt_ingestao == "" else datetime.strptime(dt_ingestao, format_timestamp)

# Pega o horário atual e tira 3 horas para converter para BRT (UTC -3)
# Tira 5 minutos por conta de um limite da API
dt_fim = dt_ingestao - timedelta(hours=3) - timedelta(minutes=5)
dt_inicio = dt_fim - timedelta(days=1)

dt_inicio_api_format = dt_ingestao - timedelta(days=5)
dt_fim_api_format = dt_ingestao
dt_inicio_api = dt_inicio_api_format.strftime(format_timestamp_api)
dt_fim_api = dt_fim_api_format.strftime(format_timestamp_api)

# Força as horas, minutos e segundos em 0
dt_inicio = datetime(dt_inicio.year, dt_inicio.month, dt_inicio.day, 0, 0, 0)

# Separa as datas no formato UNIX
dt_fim_unix = int(mktime(dt_fim.timetuple()))
dt_inicio_unix = int(mktime(dt_inicio.timetuple()))

# dt_inicio_unix = 1546300800

# Transforma as datas em STRING utilizando um formato de timestamp
dt_fim = dt_fim.strftime(format_timestamp)
dt_inicio = dt_inicio.strftime(format_timestamp)

print(f"Data ínicio do filtro API: {dt_inicio_api} | Data fim do filtro API: {dt_fim_api}")
print(f"dt_inicio: {dt_inicio} | unix: {dt_inicio_unix}")
print(f"dt_fim   : {dt_fim} | unix: {dt_fim_unix}")

In [ ]:
api_token = dbutils.secrets.get("scope-vault-data", "pipedrive-api-token")
domain = 'api.pipedrive.com/v1'
params = {'api_token': f'{api_token}'}
threads = 10
baseUrl = f"https://{domain}"

In [ ]:
df_param = fn_ConsultaJdbc("""
    SELECT *
    FROM ctl.ADF_Parametro_Carga_API pca
    WHERE pca.fl_ativo = 1
    and nm_Sistema = 'pipedrive'
""")

display(df_param)
data_param = df_param.collect()

In [ ]:
def get_tables(param, baseUrl, dt_inicio_unix, dtIngestao, location_landing):
    end_point_page = None
    try:
        format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
        dtIngestao = str(dtIngestao)
        if param.vl_Ultimo_Incremento is not None:
            vlUltimoIncremento = datetime.strptime(param.vl_Ultimo_Incremento, format_timestamp)
            vlUltimoIncremento = int(mktime(vlUltimoIncremento.timetuple()))
        else:
            vlUltimoIncremento = dt_inicio_unix

        vlUltimoIncremento = str(vlUltimoIncremento)

        # Declarando as variáveis
        pagina = 1
        cursor = None
        paginacao = False
        inicio = time.time()

        if param.vl_Schedule_Carga != 0:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
        else:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()

        print(f"Tabela {param.nm_Item_Origem}! Começando carga...")

        data_list = []

        # Pega o endpoint que utilizará no request
        # Verificação para casos especiais que salvam a última URL que processou
        # Faz um replace passando a página 0 para a primeira requisição, no caso da tabela activities utiliza o cursor que, para a primeira requisição, não é utilizado!
        if param.ds_Custom_Field is None:
            end_point_page = f"{baseUrl}{param.ds_Url.replace('@pagina', '0').replace('@dtInicio','').replace('@dtFim','')}"
        else:
            end_point_page = param.ds_Custom_Field
            if param.nm_Item_Origem == 'activities':
                end_point_page = end_point_page.replace('@dtInicio',dt_inicio_api).replace('@dtFim',dt_fim_api).replace('@pagina','0')      

        # Envia a primeira requisição
        req = requests.get(end_point_page,params=params)
        last_endpoint = end_point_page

        # Verifica se a primeira requisição teve erro
        if req.status_code != 200:
            print("\n"+f"Tabela {param.nm_Item_Origem} com erro na primeira requisição!")
        # Caso a response da requisição seja 200 (sucesso) 
        else:
            # Armazena o conteúdo da API na variável data em formato JSON
            data = req.json()

            data_keys = list(data.keys())

            if 'data' in data_keys:
                if data['data'] == None:
                    return

            if 'additional_data' in data_keys:
                if 'next_cursor' in list(data['additional_data'].keys()):
                    if data['additional_data']['next_cursor'] != None:
                        cursor = data['additional_data']['next_cursor']
                        try:
                            end_point_page = f"{baseUrl}{param.ds_Url.replace('@cursor', cursor)}"
                        except:
                            raise Exception(f'Não foi possível encontrar a próxima página para a tabela --> {param.ds_Nome_Arquivo_Landing}\nPágina --> {pagina}')
                elif 'pagination' in list(data['additional_data'].keys()):
                    if 'more_items_in_collection' in list(data['additional_data']['pagination'].keys()):
                        if data['additional_data']['pagination']['more_items_in_collection'] == True:
                            paginacao = data['additional_data']['pagination']['more_items_in_collection']
                            param_start = str(data['additional_data']['pagination']['next_start'])
                            try:
                                end_point_page = f"{baseUrl}{param.ds_Url.replace('@pagina', param_start)}"
                                if param.nm_Item_Origem == 'activities':
                                    if param.ds_Custom_Field is not None:
                                        end_point_page = param.ds_Custom_Field.replace('@dtInicio',dt_inicio_api).replace('@dtFim',dt_fim_api).replace('@pagina',param_start)
                                    else:
                                        end_point_page = f"{baseUrl}{param.ds_Url.replace('@dtInicio','').replace('@dtFim','').replace('@pagina',param_start)}"
                            except:
                                raise Exception(f'Não foi possível encontrar a próxima página para a tabela --> {param.ds_Nome_Arquivo_Landing}\nPágina --> {pagina}')

            # Salva os dados na lista
            if param.nm_Item_Origem == 'goals':
                data_list.extend(data['data']['goals'])
            else:
                data_list.extend(data['data'])

            if len(data_list) == 0: # caso a quantidade de registros seja 0 (vazio), encerra a execução para evitar erros no LOG
                return

        # Faz requisições até que o campo "next-page" da API venha com valor Null, significa que todas as páginas foram percorridas
        while paginacao == True or cursor != None:
            cursor = None
            paginacao = False
            pagina += 1

            # Envia a requisição para a página seguinte
            req = requests.get(end_point_page,params=params)
            last_endpoint = end_point_page

            # Em caso de erro entra no laço
            if req.status_code != 200:
                print("\n"+f"Tabela {param.nm_Item_Origem} com erro na requisição da página {pagina}")
            else:
                data = req.json()

                data_keys = list(data.keys())

                if 'data' in data_keys:
                    if data['data'] == None:
                        break

                if 'additional_data' in data_keys:
                    if 'next_cursor' in list(data['additional_data'].keys()):
                        if data['additional_data']['next_cursor'] != None:
                            cursor = data['additional_data']['next_cursor']
                            try:
                                end_point_page = f"{baseUrl}{param.ds_Url.replace('@cursor', cursor)}"
                            except:
                                raise Exception(f'Não foi possível encontrar a próxima página para a tabela --> {param.ds_Nome_Arquivo_Landing}\nPágina --> {pagina}')
                    elif 'pagination' in list(data['additional_data'].keys()):
                        if 'more_items_in_collection' in list(data['additional_data']['pagination'].keys()):
                            if data['additional_data']['pagination']['more_items_in_collection'] == True:
                                paginacao = data['additional_data']['pagination']['more_items_in_collection']
                                param_start = str(data['additional_data']['pagination']['next_start'])
                                try:
                                    end_point_page = f"{baseUrl}{param.ds_Url.replace('@pagina', param_start)}"
                                    if param.nm_Item_Origem == 'activities':
                                        if param.ds_Custom_Field is not None:
                                            end_point_page = param.ds_Custom_Field.replace('@dtInicio',dt_inicio_api).replace('@dtFim',dt_fim_api).replace('@pagina',param_start)
                                        else:
                                            end_point_page = f"{baseUrl}{param.ds_Url.replace('@dtInicio','').replace('@dtFim','').replace('@pagina',param_start)}"
                                except:
                                    raise Exception(f'Não foi possível encontrar a próxima página para a tabela --> {param.ds_Nome_Arquivo_Landing}\nPágina --> {pagina}')
                
                if param.nm_Item_Origem == 'goals':
                    data_list.extend(data['data']['goals'])
                else:
                    data_list.extend(data['data'])

        fim = time.time()

        if param.ds_Custom_Field is not None and param.nm_Item_Origem != 'activities':
            fn_AtualizaCustomField_API(param.id_Parametro_Carga_API, last_endpoint)
        
        # Subtrai a variável "início" pela "fim" para obter o tempo total de execução da tabela e armazena na váriavel "tempo_exec"
        tempo_exec = fim - inicio

        print("\n"+f"A tabela {param.nm_Item_Origem} teve {pagina} páginas carregadas com êxito!")
        print(f"Tamanho total de registros {len(data_list)}")
        print(f"O tempo de execução da tabela {param.nm_Item_Origem} foi {tempo_exec}")

        # PROCESSO DE CARGA DOS DADOS NO BLOB
        fn_SaveJson(data_list, dir_landing, str(param.ds_Nome_Arquivo_Landing).lower())

        fn_AtualizaUltimoIncremento_API(param.id_Parametro_Carga_API, dtIngestao)

        ## LOG SUCESSO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "execUrl": execUrl}

        fn_LogSucceeded(parametros, dt_ingestao.strftime(format_log))

    except Exception as error:
        ## LOG ERRO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '

        if hasattr(error, 'code'): # captura o código do erro, caso possua o atributo 'code'
            error_code = error.code
        else:
            error_code = 'NULL'
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "cd_erro": error_code, "erro": str(error), "execUrl": execUrl}

        fn_LogFailed(parametros, dt_ingestao.strftime(format_log))

In [ ]:
# Cria uma lista para armazenar todas as tarefas
tasks = []

# Cria uma instância do ThreadPoolExecutor com threads definidas (max_workers)
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    # Percorre todas as pastas do diretório de origem
    for row in data_param:
        # Executa a função fn_StreamFromFolder_csv em uma thread do ThreadPoolExecutor
        task = executor.submit(get_tables, *(row, baseUrl, dt_inicio_unix, dt_fim, location_landing))
                               
        # Adiciona a tarefa à lista de tarefas
        tasks.append(task)

# Aguarda a conclusão de todas as tarefas
_ = concurrent.futures.wait(tasks, return_when='ALL_COMPLETED')

In [ ]:
for task in tasks:
    try:
        print(task.result(),'\n')
    except Exception as error:
        print(error)